In [ ]:
%pip install beacon-api --upgrade


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from beacon_api import *
import numpy as np
client = Client("https://beacon-argo.maris.nl", jwt_token="<your_jwt_token_here>")

Connected to: https://beacon-argo.maris.nl/ server successfully


In [12]:
tables = client.list_tables()
tables

{'default': <beacon_api.table.DataTable at 0x7f864b4499d0>,
 'argo-core': <beacon_api.table.DataTable at 0x7f8649db1e20>,
 'argo-bgc': <beacon_api.table.DataTable at 0x7f8649e06990>}

In [13]:
tables['default'].get_table_schema()

__entry_key: string
DATA_TYPE: string
DATA_TYPE.long_name: string
DATA_TYPE.conventions: string
DATA_TYPE._FillValue: string
FORMAT_VERSION: string
FORMAT_VERSION.long_name: string
FORMAT_VERSION._FillValue: string
HANDBOOK_VERSION: string
HANDBOOK_VERSION.long_name: string
HANDBOOK_VERSION._FillValue: string
REFERENCE_DATE_TIME: string
REFERENCE_DATE_TIME.long_name: string
REFERENCE_DATE_TIME.conventions: string
REFERENCE_DATE_TIME._FillValue: string
DATE_CREATION: string
DATE_CREATION.long_name: string
DATE_CREATION.conventions: string
DATE_CREATION._FillValue: string
DATE_UPDATE: string
DATE_UPDATE.long_name: string
DATE_UPDATE.conventions: string
DATE_UPDATE._FillValue: string
PLATFORM_NUMBER: string
PLATFORM_NUMBER.long_name: string
PLATFORM_NUMBER.conventions: string
PLATFORM_NUMBER._FillValue: string
PROJECT_NAME: string
PROJECT_NAME.long_name: string
PROJECT_NAME._FillValue: string
PI_NAME: string
PI_NAME.long_name: string
PI_NAME._FillValue: string
STATION_PARAMETERS: string
S

In [ ]:
# METADATA
from datetime import datetime, timezone
dt_str = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

query_builder = tables['default'].query()

query_builder = (
    query_builder
    .add_select_column("__entry_key")
    
    .add_select(SelectLiteral("BEACON_ARGO", alias="SOURCE_BDI"))
    .add_select(SelectLiteral("2026-01-14T00:00:00Z", "COMMON_BDI_SNAPSHOT_DATE"))
    .add_select(SelectLiteral("1.5.0", "COMMON_BDI_MONOLITH_VERSION"))
    .add_select(SelectLiteral(dt_str, "COMMON_HARMONIZATION_DATE"))
    
    .add_select(SelectFunction("concat", [SelectColumn("PLATFORM_NUMBER"), SelectLiteral("_"), SelectColumn("CYCLE_NUMBER"), SelectLiteral("_"), SelectColumn("DIRECTION")], alias="SOURCE_BDI_DATASET_ID"))
    .add_select(SelectColumn("PLATFORM_NUMBER"))
    .add_select(SelectColumn("DIRECTION"))
    .add_select(SelectColumn("CYCLE_NUMBER"))
    .add_select(SelectColumn("WMO_INST_TYPE"))
    .add_select(SelectColumn(".featureType"))
    .add_select(SelectColumn(".featureType", alias="COMMON_FEATURE_TYPE"))
    .add_select(SelectFunction("concat", [SelectColumn("__entry_key"), SelectColumn("CYCLE_NUMBER"), SelectColumn("DIRECTION")], "COMMON_ODV_TAG"))
    .add_select(SelectFunction("map_argo_platform_l06", [Functions.try_cast_to_type(SelectLiteral(0), np.int64, "platform_l06")], alias="COMMON_PLATFORM_L06"))
)

This is an integer dtype: int64


In [15]:
# TIME, LOCATION, DEPTH
query_builder = (
    query_builder
    .add_select(SelectColumn("JULD"))
    .add_select(SelectColumn("JULD_QC"))
    .add_select(SelectColumn("JULD", alias="COMMON_TIME"))
    .add_select(SelectColumn("JULD_QC", alias="COMMON_TIME_QC"))
    
    .add_select(SelectColumn("LATITUDE"))
    .add_select(SelectColumn("LATITUDE.standard_name"))
    .add_select(SelectColumn("LATITUDE.units"))
    .add_select(SelectColumn("LATITUDE", alias="COMMON_LATITUDE"))
    .add_select(SelectLiteral("degrees_north", "COMMON_LATITUDE_UNITS"))
    .add_select(SelectLiteral("Latitude", "COMMON_LATITUDE_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ALATZZ01", "COMMON_LATITUDE_P01"))
    .add_select(SelectLiteral("SDN:P06::DEGN", "COMMON_LATITUDE_P06"))
    
    .add_select(SelectColumn("LONGITUDE"))
    .add_select(SelectColumn("LONGITUDE.standard_name"))
    .add_select(SelectColumn("LONGITUDE.units"))
    .add_select(SelectColumn("LONGITUDE", alias="COMMON_LONGITUDE"))
    .add_select(SelectLiteral("degrees_east", "COMMON_LONGITUDE_UNITS"))
    .add_select(SelectLiteral("Longitude", "COMMON_LONGITUDE_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ALONZZ01", "COMMON_LONGITUDE_P01"))
    .add_select(SelectLiteral("SDN:P06::DEGE", "COMMON_LONGITUDE_P06"))
    
    .add_select(SelectColumn("POSITION_QC"))
    .add_select(SelectColumn("POSITION_QC", alias="COMMON_POSITION_QC"))
    
    .add_select_column("PRES")
    .add_select_column("PRES_ADJUSTED")
    .add_select_column("PRES.units")
    .add_select_column("PRES_ADJUSTED.units")
    .add_select_column("PRES_QC")
    .add_select_column("PRES_ADJUSTED_QC")
    .add_select_coalesced(["PRES_ADJUSTED", "PRES"], "COMMON_ORIGIN_DEPTH")
    .add_select_coalesced(["PRES_ADJUSTED_QC", "PRES_QC"], "COMMON_ORIGIN_DEPTH_QC")
    
    .add_select(SelectColumn("PRES.units", "COMMON_ORIGIN_DEPTH_UNITS"))
    .add_select(SelectLiteral("PRES.standard_name", "COMMON_ORIGIN_DEPTH_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::PRESPR01", "COMMON_ORIGIN_DEPTH_P01"))
    .add_select(SelectLiteral("SDN:P06::UPDB", "COMMON_ORIGIN_DEPTH_P06"))

    .add_select(SelectFunction("pressure_to_depth_teos_10", [SelectFunction("coalesce", [SelectColumn("PRES_ADJUSTED"), SelectColumn("PRES")]), SelectColumn("LATITUDE")], "COMMON_DEPTH"))
    .add_select(SelectFunction("coalesce", [SelectColumn("PRES_ADJUSTED_QC"), SelectColumn("PRES_QC")], "COMMON_DEPTH_QC"))
    .add_select(SelectLiteral("m", "COMMON_DEPTH_UNITS"))
    .add_select(SelectLiteral("depth", "COMMON_DEPTH_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ADEPZZ01", "COMMON_DEPTH_P01"))
    .add_select(SelectLiteral("SDN:P06::ULAA", "COMMON_DEPTH_P06"))
)

In [16]:
# TEMPERATURE
query_builder = (
    query_builder
    .add_select(SelectColumn("TEMP"))
    .add_select(SelectColumn("TEMP_QC"))
    .add_select(SelectColumn("TEMP_ADJUSTED"))
    .add_select(SelectColumn("TEMP_ADJUSTED_QC"))
    .add_select_coalesced(["TEMP_ADJUSTED", "TEMP"], "COMMON_TEMPERATURE")
    .add_select_coalesced(["TEMP_ADJUSTED_QC", "TEMP_QC"], "COMMON_TEMPERATURE_QC")
    .add_select(SelectLiteral("SDN:P01::TEMPPR01", "COMMON_TEMPERATURE_P01"))
    .add_select(SelectLiteral("SDN:P06::UPAA", "COMMON_TEMPERATURE_P06"))
    .add_select(SelectLiteral("sea_water_temperature", "COMMON_TEMPERATURE_STANDARD_NAME"))
    .add_select(SelectLiteral("degree_C", "COMMON_TEMPERATURE_UNITS"))
    .add_select(SelectFunction("map_argo_instrument_l05", [Functions.try_cast_to_type(SelectLiteral(0), np.int64, "temp_l05")], "COMMON_TEMPERATURE_L05"))
    .add_select(SelectFunction("map_wmo_instrument_type_l22", [SelectFunction("btrim",[SelectColumn("WMO_INST_TYPE")])], "COMMON_TEMPERATURE_L22"))
)

This is an integer dtype: int64


In [17]:
# Salinity
query_builder = (
    query_builder
    .add_select(SelectColumn("PSAL"))
    .add_select(SelectColumn("PSAL_QC"))
    .add_select(SelectColumn("PSAL_ADJUSTED"))
    .add_select(SelectColumn("PSAL_ADJUSTED_QC"))
    .add_select_coalesced(["PSAL_ADJUSTED", "PSAL"], "COMMON_SALINITY")
    .add_select_coalesced(["PSAL_ADJUSTED_QC", "PSAL_QC"], "COMMON_SALINITY_QC")
    .add_select(SelectLiteral("SDN:P01::PSLTZZ01", "COMMON_SALINITY_P01"))
    .add_select(SelectLiteral("SDN:P06::UUUU", "COMMON_SALINITY_P06"))
    .add_select(SelectLiteral("sea_water_practical_salinity", "COMMON_SALINITY_STANDARD_NAME"))
    .add_select(SelectLiteral("dimensionless", "COMMON_SALINITY_UNITS"))
    .add_select(SelectFunction("map_argo_instrument_l05", [Functions.try_cast_to_type(SelectLiteral(0), np.int64, "psal_l05")], "COMMON_SALINITY_L05"))
    .add_select(SelectFunction("map_wmo_instrument_type_l22", [SelectFunction("btrim",[SelectColumn("WMO_INST_TYPE")])], "COMMON_SALINITY_L22"))
)

This is an integer dtype: int64


In [ ]:
import os 
import json

# Make output dir
output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)
# for each year between 1950 and 2025
for year in range(1950, 2026):
    print(f"Processing year {year}")
    query_builder.filters = []
    query_builder = query_builder.add_range_filter("COMMON_TIME", f"{year}-01-01T00:00:00Z", f"{year}-12-31T23:59:59Z")
    query_builder = (
        query_builder.add_filter(OrFilter([
            IsNotNullFilter("TEMP"),
            IsNotNullFilter("PSAL"),
        ]))
    )
    
    output_file = os.path.join(output_dir, f"argo_{year}.parquet")
    try:
        query_builder.to_parquet(output_file)
        print(f"Saved to {output_file}")
    except Exception as e:
        print(f"Failed to process year {year}: {e}")

Processing year 1950
Running query: {"from": "default", "select": [{"column": "__entry_key", "alias": null}, {"value": "BEACON_ARGO", "alias": "SOURCE_BDI"}, {"function": "concat", "args": [{"column": "PLATFORM_NUMBER", "alias": null}, {"value": "_", "alias": null}, {"column": "CYCLE_NUMBER", "alias": null}, {"value": "_", "alias": null}, {"column": "DIRECTION", "alias": null}], "alias": "SOURCE_BDI_DATASET_ID"}, {"column": "PLATFORM_NUMBER", "alias": null}, {"column": "DIRECTION", "alias": null}, {"column": "CYCLE_NUMBER", "alias": null}, {"column": "WMO_INST_TYPE", "alias": null}, {"column": ".featureType", "alias": null}, {"column": ".featureType", "alias": "COMMON_FEATURE_TYPE"}, {"function": "concat", "args": [{"column": "__entry_key", "alias": null}, {"column": "CYCLE_NUMBER", "alias": null}, {"column": "DIRECTION", "alias": null}], "alias": "COMMON_ODV_TAG"}, {"function": "map_argo_platform_l06", "args": [{"function": "try_arrow_cast", "args": [{"value": 0, "alias": null}, {"val